## t2

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# 读取数据并生成特征
def generate_features(df):
    df = df.copy()
    if 'I' in df.columns:
        df.drop(columns=['I', 'x_squared', 'x_3', 'I_x', 'I_intercept'], inplace=True, errors='ignore')
    df['I'] = df['x'].apply(lambda x: 1 if 13 <= x <= 17 else 0)
    df['x_squared'] = df['x']**2
    df['x_3'] = df['x']**3
    df['I_x'] = df['I'] * df['x']
    df['I_intercept'] = df['I']
    return df[['x', 'x_squared', 'x_3', 'I_intercept', 'I_x']]

# 加载原始数据
df_all = pd.read_excel('待拟合函数.xlsx', sheet_name='Sheet1')
X_all = generate_features(df_all.copy())
y_all = df_all['y']


#### mse


In [ ]:

# 必须保留的点（x=8和x=24）
must_keep_indices = df_all[(df_all['x'] == 8) | (df_all['x'] == 24)].index.tolist()
current_df = df_all.copy()
removed_order = []
r2_history = []
# 初始模型（使用所有点）
model = LinearRegression()
model.fit(generate_features(current_df), current_df['y'])
initial_r2 = mean_absolute_error(y_all, model.predict(X_all))   #r2_score


r2_history.append(initial_r2)
print(f"初始R²: {initial_r2:.4f}")

k=-1
# 逐步删除点
while True:
    best_r2 = -np.inf*k
    best_point = None
    candidates = [idx for idx in current_df.index if idx not in must_keep_indices]
    
    if not candidates:
        print("无法继续删除，所有非必须点已移除。")
        break
    
    # 遍历每个候选点，找出删除后R²最大的情况
    for candidate in candidates:
        temp_df = current_df.drop(candidate)
        X_temp = generate_features(temp_df)
        y_temp = temp_df['y']
        
        try:
            model = LinearRegression().fit(X_temp, y_temp)
            pred_all = model.predict(X_all)
            r2 = mean_absolute_error(y_all, pred_all)
        except:
            r2 = -np.inf*k  # 处理无法拟合的情况
        
        if r2 < best_r2:
            best_r2 = r2
            best_point = candidate
    
    # 记录结果并删除点
    if best_r2 > 0.04:
        print("停止删除。")
        break
    
    removed_order.append(current_df.loc[best_point, 'x'])
    current_df = current_df.drop(best_point)
    r2_history.append(best_r2)
    print(f"删除x={removed_order[-1]}, 当前R²: {best_r2:.4f}, 剩余点数: {len(current_df)}")

# 输出结果
print("\n删除顺序及R²变化：")
for x, r2 in zip(removed_order, r2_history[1:]):
    print(f"删除点 x={x:<3} → R²={r2:.9f}")

minimal_points = current_df['x'].tolist()
print(f"\n最少需要 {len(minimal_points)} 个点：{sorted(minimal_points)}")
print(f"最终R²: {r2_history[-1]:.4f}")

初始R²: 0.0338
删除x=23, 当前R²: 0.0212, 剩余点数: 16
删除x=16, 当前R²: 0.0212, 剩余点数: 15
删除x=15, 当前R²: 0.0212, 剩余点数: 14
删除x=20, 当前R²: 0.0212, 剩余点数: 13
删除x=17, 当前R²: 0.0212, 剩余点数: 12
删除x=9, 当前R²: 0.0212, 剩余点数: 11
删除x=18, 当前R²: 0.0212, 剩余点数: 10
删除x=22, 当前R²: 0.0212, 剩余点数: 9
删除x=19, 当前R²: 0.0212, 剩余点数: 8
删除x=12, 当前R²: 0.0212, 剩余点数: 7
删除x=10, 当前R²: 0.0212, 剩余点数: 6
停止删除。

删除顺序及R²变化：
删除点 x=23  → R²=0.021176471
删除点 x=16  → R²=0.021176471
删除点 x=15  → R²=0.021176471
删除点 x=20  → R²=0.021176471
删除点 x=17  → R²=0.021176471
删除点 x=9   → R²=0.021176471
删除点 x=18  → R²=0.021176471
删除点 x=22  → R²=0.021176471
删除点 x=19  → R²=0.021176471
删除点 x=12  → R²=0.021176471
删除点 x=10  → R²=0.021176471

最少需要 6 个点：[8, 11, 13, 14, 21, 24]
最终R²: 0.0212


: 

#### r^2

In [3]:

# 必须保留的点（x=8和x=24）
must_keep_indices = df_all[(df_all['x'] == 8) | (df_all['x'] == 24)].index.tolist()
current_df = df_all.copy()
removed_order = []
r2_history = []
# 初始模型（使用所有点）
model = LinearRegression()
model.fit(generate_features(current_df), current_df['y'])
initial_r2 = r2_score(y_all, model.predict(X_all))   #r2_score


r2_history.append(initial_r2)
print(f"初始R²: {initial_r2:.4f}")

k=1
# 逐步删除点
while True:
    best_r2 = -np.inf*k
    best_point = None
    candidates = [idx for idx in current_df.index if idx not in must_keep_indices]
    
    if not candidates:
        print("无法继续删除，所有非必须点已移除。")
        break
    
    # 遍历每个候选点，找出删除后R²最大的情况
    for candidate in candidates:
        temp_df = current_df.drop(candidate)
        X_temp = generate_features(temp_df)
        y_temp = temp_df['y']
        
        try:
            model = LinearRegression().fit(X_temp, y_temp)
            pred_all = model.predict(X_all)
            r2 = r2_score(y_all, pred_all)
        except:
            r2 = -np.inf*k  # 处理无法拟合的情况
        
        if r2 > best_r2:
            best_r2 = r2
            best_point = candidate
    
    # 记录结果并删除点
    if best_r2 < 0.99:
        print("停止删除。")
        break
    
    removed_order.append(current_df.loc[best_point, 'x'])
    current_df = current_df.drop(best_point)
    r2_history.append(best_r2)
    print(f"删除x={removed_order[-1]}, 当前R²: {best_r2:.4f}, 剩余点数: {len(current_df)}")

# 输出结果
print("\n删除顺序及R²变化：")
for x, r2 in zip(removed_order, r2_history[1:]):
    print(f"删除点 x={x:<3} → R²={r2:.8f}")

minimal_points = current_df['x'].tolist()
print(f"\n最少需要 {len(minimal_points)} 个点：{sorted(minimal_points)}")
print(f"最终R²: {r2_history[-1]:.4f}")

初始R²: 1.0000
删除x=14, 当前R²: 1.0000, 剩余点数: 16
删除x=10, 当前R²: 1.0000, 剩余点数: 15
删除x=9, 当前R²: 1.0000, 剩余点数: 14
删除x=16, 当前R²: 1.0000, 剩余点数: 13
删除x=18, 当前R²: 1.0000, 剩余点数: 12
删除x=19, 当前R²: 1.0000, 剩余点数: 11
删除x=15, 当前R²: 1.0000, 剩余点数: 10
删除x=11, 当前R²: 1.0000, 剩余点数: 9
删除x=20, 当前R²: 1.0000, 剩余点数: 8
删除x=22, 当前R²: 1.0000, 剩余点数: 7
删除x=23, 当前R²: 1.0000, 剩余点数: 6
停止删除。

删除顺序及R²变化：
删除点 x=14  → R²=0.99998992
删除点 x=10  → R²=0.99998992
删除点 x=9   → R²=0.99998992
删除点 x=16  → R²=0.99998992
删除点 x=18  → R²=0.99998992
删除点 x=19  → R²=0.99998992
删除点 x=15  → R²=0.99998992
删除点 x=11  → R²=0.99998991
删除点 x=20  → R²=0.99998974
删除点 x=22  → R²=0.99998736
删除点 x=23  → R²=0.99998609

最少需要 6 个点：[8, 12, 13, 17, 21, 24]
最终R²: 1.0000
